## Goal
We want to explore how better is the performance of the system of generating a draft with a multiple step process rather than a single step process

### Initial Variables

In [4]:
import sys 
import os
import json_tricks as json
import time
from dotenv import load_dotenv
load_dotenv('../../backend/.env')
sys.path.append("../../")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Generater import Generater
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import Email


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
generator = Generater(ollama_client, anyllm_client)
path_to_enquiry = "../../dataset/fake_email_1.json"
with open(path_to_enquiry, "r") as f:
    enquiry = json.load(f)

enquiry_text = enquiry["email"]
subject = enquiry_text.split("\n\n")[0]
body = '\n'.join(enquiry_text.split("\n\n")[1:])

Performing health check on http://localhost:11434
Ollama is running
Health check passed


### Single Step Process

In [2]:

def single_shot_prompt(email: Email):
    prompt = f""" You are David Frischer, the program administrator for the UCL Software Engineering program.
            You have received the following enquiry from a student:
            Subject: {email.subject}
            Body: {email.body}
            
           
            Draft a response to the student's email.
            """
    workspace_slug =  anyllm_client.get_workspace_slug("General")
    thread = anyllm_client.new_thread(workspace_slug, "tmp-thread")
    thread_slug = thread["slug"]
    start_time = time.time()
    response = anyllm_client.chat_with_thread(workspace_slug, thread_slug, prompt)
    total_time = time.time() - start_time
    anyllm_client.delete_thread(workspace_slug, thread_slug)
    return response, total_time

### Response format

## Mutiple Step Process

In [3]:
def multi_step_process(email:Email):
    start_time = time.time()
    response = generator.single_run_reply_to_email(email)
    response_time = time.time() - start_time
    
    return response, response_time
    

## Generate for multiple drafts

In [4]:

path_to_enquiries = "../../dataset"

counter = 0
results_folder = "./results"
os.makedirs(results_folder, exist_ok=True)

already_processed = set()
for results_file in os.listdir(results_folder):
    if results_file.endswith(".json"):
        already_processed.add(results_file.replace("results_", ""))
        
for enquiry_path in os.listdir(path_to_enquiries):
    if enquiry_path in already_processed:
        print(f"Already processed {enquiry_path}")
        continue
    if "fake" not in enquiry_path and not enquiry_path.endswith(".json"):
        print(f"Skipping {enquiry_path}")
        continue
      
    filepath = os.path.join(results_folder, f"results_{enquiry_path}")
    results = {}
    try:
        if not enquiry_path.endswith(".json") or not "fake" in enquiry_path:
            continue
        print(f"Processing {enquiry_path}")
        with open(os.path.join(path_to_enquiries, enquiry_path), "r") as f:
            enquiry = json.load(f)
            
        email = enquiry["email"]
        if "subject:" in email.lower():
            subject = email.split("\n")[0].replace("Subject:", "").strip()
            body = "\n".join(email.split("\n")[1:])
        else:
            subject = "Subject not found"
            body = email
        
        email = Email(subject=subject, body=body, id=counter)
        counter += 1
        
        response_single, response_time_single = single_shot_prompt(email)
        response_multi, response_time_multi = multi_step_process(email)
        
        results = {"single_shot": {"response": response_single, "response_time": response_time_single},
                    "multi_step": {"response": response_multi, "response_time": response_time_multi}}

        with open(filepath, "w") as f:
            json.dump(results, f)
            
    except Exception as e:
        print(f"Error processing {enquiry_path}")
        print(e)
        continue    

print("Done")


Already processed fake_email_10.json
Already processed fake_email_undergrad_13.json
Already processed fake_email_undergrad_6.json
Already processed fake_email_3.json
Already processed fake_email_2.json
Already processed fake_email_undergrad_7.json
Already processed fake_email_undergrad_12.json
Already processed fake_email_11.json
Already processed fake_email_undergrad_0.json
Already processed fake_email_undergrad_19.json
Already processed fake_email_5.json
Already processed fake_email_16.json
Already processed fake_email_undergrad_15.json
Already processed fake_email_9.json
Already processed fake_email_8.json
Already processed fake_email_undergrad_14.json
Already processed fake_email_17.json
Skipping __pycache__
Already processed fake_email_4.json
Already processed fake_email_undergrad_18.json
Already processed fake_email_undergrad_1.json
Already processed fake_email_7.json
Skipping bank_questions.py
Already processed fake_email_undergrad_2.json
Already processed fake_email_18.json
Alr

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: new_thread in workspace: general
Total Sim Distance: 0.3058445304632187
Total Sim Distance: 0.3207991123199463
['Question: As a prospective graduate student, can I apply for multiple programs simultaneously? If yes, are there any specific procedures or requirements I need to follow?\nAnswer: Based on the provided context from UCL\'s website, the answer to your question is:\n\nYes, as a prospective graduate student, you may submit applications for a maximum of two taught graduate programmes for entry to the 2024/25 academic year. The applications must be for different programs. Please note that if an application fee is charged, it will be charged per application.\n\nThis information can be found in the "Applications" section of UCL\'s Frequently Asked Questions (FAQs) for prospective students applying to graduate taught and research programmes. \n-------\n', "Question: I am interested in gaining accreditation for prior learning as part of my MSc applicatio

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Processing fake_email_undergrad_11.json
Creating new thread with name: tmp-thread in workspace: general


...

Processing fake_email_undergrad_10.json
Creating new thread with name: tmp-thread in workspace: general


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: new_thread in workspace: general
Total Sim Distance: 0.31222349405288696
Total Sim Distance: 0.3378884643316269
Total Sim Distance: 0.3311948627233505
['Question: Can I still apply for an undergraduate programme at UCL? If so, would it be possible to receive a conditional offer based on my pending exam results?\nAnswer: Based on the provided contexts from University College London (UCL), here\'s an answer to your question:\n\nAccording to Context 3, which is related to undergraduate programmes, it states that "Resits are generally considered for most of our programmes. Any programmes that do not consider resits will outline this on their department webpages." Since you\'re interested in applying for the Computer Science programme at UCL, and you\'re an adult learner returning to education, it\'s likely that your case would be considered.\n\nRegarding your question about receiving a conditional offer based on your pending exam results, Context 2 provides s

[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Done


In [13]:
import json as simplejson
from random import randint

def results_into_markodown():
    results_folder = "./results"
    mardown_folder = "./markdown"
    os.makedirs(mardown_folder, exist_ok=True)
    markdown = ""
    answer = {}
    for results_file in os.listdir(results_folder):
        if results_file.endswith(".json"):
            with open(os.path.join(results_folder, results_file), "r") as f:
                results = simplejson.load(f)
                
            random_number = randint(1, 2)
            single_shot_response = results['single_shot']['response']['textResponse']
            multi_step_response = results['multi_step']['response']['generated_draft_email']
            filename = results_file.replace("results_", "").replace("json", "md")
            markdown = f"# {filename}\n\n"
            if random_number == 1:
                markdown += f"## A\n\n"
                markdown += f"{single_shot_response}\n\n"
                markdown += f"## B\n\n"
                markdown += f"{multi_step_response}\n\n"
                answer[filename] = {"A": 'single_shot_response', "B": 'multi_step_response'}
            else:
                markdown += f"## A\n\n"
                markdown += f"{multi_step_response}\n\n"
                markdown += f"## B\n\n"
                markdown += f"{single_shot_response}\n\n"
                answer[filename] = {"A": 'multi_step_response', "B": 'single_shot_response'}
            
            
            markdown += "\n\n"
            
            with open(os.path.join(mardown_folder, results_file.replace("results_", "").replace("json", "md")), "w") as f:
                f.write(markdown)
    with open(os.path.join(mardown_folder, "answers.json"), "w") as f:
        simplejson.dump(answer, f)
        


In [5]:
import json as simplejson

def average_time():
    results_folder = "./results"
    total_single_shot = 0
    total_multi_step = 0
    total = 0
    for results_file in os.listdir(results_folder):
        if results_file.endswith(".json"):
            with open(os.path.join(results_folder, results_file), "r") as f:
                results = simplejson.load(f)
            total_single_shot += results['single_shot']['response_time']
            total_multi_step += results['multi_step']['response_time']
            total += 1
    print(f"Average single shot time: {total_single_shot/total}")
    print(f"Average multi step time: {total_multi_step/total}")
    
average_time()

Average single shot time: 51.24523681998253
Average multi step time: 174.0607920050621
